In [5]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
pi = 3.141592653589793238
w = 2*pi / 10
eps = 0.1
A = 0.1
def f(x, t):
    a = eps * np.sin(w * t)
    b = 1 - eps * 2 * np.sin(w * t)
    return a * x * x + b * x
def df(x, t):
    a = eps * np.sin(w * t)
    b = 1 - eps * 2 * np.sin(w * t)
    return 2 * a * x + b 

def vx(x, y, t):
    return -pi * A * np.sin(pi * f(x,t)) * np.cos(pi * y)
def vy(x, y, t):
    return pi * A * np.cos(pi * f(x,t)) * np.sin(pi * y) * df(x, t)


In [6]:
import time

times = 200
tmax = 20
dt = tmax / times
h = dt
x_points = 400
y_pints = int(x_points / 2)

D = 4*2 / x_points

x = np.linspace(-D,2 + D,x_points)
y = np.linspace(-D,1 + D,y_pints)
X, Y = np.meshgrid(x, y)

#1361
nx = 400
ny = nx


resX= 2. / nx
resY = 1. / ny

uu = np.zeros((times + 2, y_pints, x_points))
vv = np.zeros((times + 2, y_pints, x_points))

for i in range(0, times + 2):
    uu[i, :, :] = vx(X, Y, i*dt)
    vv[i, :, :] = vy(X, Y, i*dt)
    

def k_i(x, y, t, field):
    x_0 = (x / resX).astype(np.int)
    y_0 = (y / resY).astype(np.int)
    t_0 = int(t * times / tmax)
    
    x_d = (x /resX - x_0)
    y_d = (y /resY - y_0)
    t_d = (t * times / tmax - t_0)
    
#     print(x_0)
#     print(field.shape)
    x_true = np.logical_and(x_0 < x_points, x_0 > -1)
    y_true = np.logical_and(y_0 < y_pints, y_0 > -1)
    xy_true = np.logical_and(x_true, y_true)
    c00 = np.zeros(x.shape)
    c00[xy_true] = field[t_0, y_0[xy_true], x_0[xy_true]] * (1 - x_d[xy_true]) + field[t_0 + 1, y_0[xy_true], x_0[xy_true]] * x_d[xy_true]
    
    x_true = np.logical_and(x_0 < x_points - 1, x_0 > -2)
    y_true = np.logical_and(y_0 < y_pints, y_0 > -1)
    xy_true = np.logical_and(x_true, y_true)
    c01 = np.zeros(x.shape)
    c01[xy_true] = field[t_0, y_0[xy_true], x_0[xy_true] + 1] * (1 - x_d[xy_true]) + field[t_0 + 1, y_0[xy_true], x_0[xy_true] + 1] * x_d[xy_true]
    
    x_true = np.logical_and(x_0 < x_points, x_0 > -1)
    y_true = np.logical_and(y_0 < y_pints - 1, y_0 > -2)
    xy_true = np.logical_and(x_true, y_true)
    c10 = np.zeros(x.shape)
    c10[xy_true] = field[t_0, y_0[xy_true] + 1, x_0[xy_true]] * (1 - x_d[xy_true]) + field[t_0 + 1, y_0[xy_true] + 1, x_0[xy_true]] * x_d[xy_true]
    
    x_true = np.logical_and(x_0 < x_points - 1, x_0 > -2)
    y_true = np.logical_and(y_0 < y_pints - 1, y_0 > -2)
    xy_true = np.logical_and(x_true, y_true)
    c11 = np.zeros(x.shape)
    c11[xy_true] = field[t_0, y_0[xy_true] + 1, x_0[xy_true] + 1] * (1 - x_d[xy_true]) + field[t_0 + 1, y_0[xy_true] + 1, x_0[xy_true] + 1] * x_d[xy_true]
    
    c0 = c00 * (1 - y_d) + c10 * y_d
    c1 = c01 * (1 - y_d) + c11 * y_d
    
    return c0 * (1 - t_d) + c1 * t_d

def rk4(x, y, t, uu, vv):
    kx1 = k_i(x, y, t, uu)
    ky1 = k_i(x, y, t, vv)
    
    xn = x + dt * kx1 / 2
    yn = y + dt * ky1 / 2
    tn = t + dt / 2
    
    kx2 = k_i(xn, yn, tn, uu)
    ky2 = k_i(xn, yn, tn, vv)
    
    xn = x + dt * kx2 / 2
    yn = y + dt * ky2 / 2
    
    kx3 = k_i(xn, yn, tn, uu)
    ky3 = k_i(xn, yn, tn, vv)
    
    xn = x + dt * kx3
    yn = y + dt * ky3
    tn = t + dt
    
    kx4 = k_i(xn, yn, tn, uu)
    ky4 = k_i(xn, yn, tn, vv)
    
    x = x + (kx1 + 2 * kx2 + 2 * kx3 + kx4) * dt / 6
    y = y + (ky1 + 2 * ky2 + 2 * ky3 + ky4) * dt / 6
    
    out = np.zeros((2, nx, ny))
    out[0, :, :] = x
    out[1, :, :] = y
    return out


In [7]:


def seed_parties(nx, ny):
        x  = np.linspace(resX, 2 - resX, nx)
        y  = np.linspace(resY, 1 - resY, ny)
        y, x = np.meshgrid(y, x)
        return np.array([x, y])

# X0m = seed_parties(nx, ny)
# print(resX)
# start = time.time()
# temp = np.zeros((2, nx, ny))
# temp = X0m
# for i in range(0, times - 2):
#     temp = rk4(temp[0,:,:], temp[1,:,:], i * dt, uu, vv)
    
# X1m = temp
# end = time.time()
# # print(X1m)
# print(-start + end)

In [8]:

# coding: utf-8

# In[9]:


import xarray as xa
import numpy as np
from numba import njit
from numba import jitclass
import numba

# In[10]:


import matplotlib as plt
from numba import types
from numba.extending import typeof_impl
# In[11]:


class data_obj:
    def __call__(self, field):
        self.values = field

        
u_ = data_obj()
u_(uu)

v_ = data_obj()
v_(vv)

x_ = data_obj()
x_(np.linspace(-D,2 + D,x_points))
y_ = data_obj()
y_(np.linspace(-D,1 + D,y_pints))
# In[12]:
#def setNxNY(Nx, Ny):
#    global N
#    N = np.array([Nx,Ny])


class norkystHandler():
    def __init__(self):
        limx = -1
        offset = 0
        limy = limx
        time_lim = 20
        time_offset = 0
        self.x = x_
        self.y = y_
        self.u = u_
        self.v = v_
        self.dims = self.u.values.shape
        offsets = np.empty(2)
        offsets[0] = self.x.values[0]
        offsets[1] = self.y.values[0]
        self.offsets = offsets
        self.time_res = dt
        self.x_res = self.x.values[1] - self.x.values[0]
        self.y_res = self.y.values[1] - self.y.values[0]
   
    def __call__(self, Nx, Ny):
        N_ = np.empty(2)
        N_[0] = Nx
        N_[1] = Ny
        self.N = N_



    def grid_of_particles(self, wx=2, wy = 1.):
        
        
        #Nx = 1*int(np.sqrt(N))
        #Ny = 1*int(np.sqrt(N))
        x  = np.linspace(self.x.values[0] + 5*self.x_res, self.x.values[-1] - 5*self.x_res, self.N[0])
        y  = np.linspace(self.y.values[0] + 5*self.y_res, self.y.values[-1] - 5*self.y_res, self.N[1])
        y, x = np.meshgrid(y, x)
        return np.array([x, y])
    

    #@njit(['float64[:,:](Norkyst, float64[:,:], float64)'], parallel=True)
    def f(self, X, t):
        ret = f_in3(X, t, self.offsets, self.x_res, self.y_res, self.time_res, self.u, self.v, self.dims, xcord=self.x, ycord=self.y)
        return ret

#@njit(parallel=True)    
def f_in(X, t, offsets, x_res, y_res, time_res, u, v, dims, xcord=None, ycord=None):
    #global offsets, u, v
        SHAPE = X.shape
        #print(SHAPE)
        X = X.reshape(2, SHAPE[1]*SHAPE[2])
        #X.reshape(2, )
        x_0 = np.zeros(X.shape)
        x_0[0,:] = (((X[0,:] - offsets[0]) / x_res).astype(int))
        x_0[1,:] = (((X[1,:] - offsets[1]) / y_res).astype(int))
        x_0 = x_0.astype(int)
        t_0 = int(t/time_res)
        
        x_d = np.zeros(X.shape)
        x_d[0,:] = ((X[0,:] - offsets[0]) / x_res - x_0[0,:])
        x_d[1,:] = ((X[1,:] - offsets[1]) / y_res - x_0[1,:])
        t_d = t/time_res - t_0

        ret = np.zeros((X.shape))
        
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        x_true = np.logical_and(x_0[0,:] < dims[2], x_0[0,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        
        c00 = np.zeros(X.shape[1])
        c00[xy_true] = u.values[t_0, x_0[1,xy_true], x_0[0,xy_true]] * (1 - x_d[0,xy_true]) + \
                            u.values[t_0 + 1, x_0[1,xy_true], x_0[0,xy_true]] * x_d[0,xy_true]
                      
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        
        c01 = np.zeros(X.shape[1])
        c01[xy_true] = u.values[t_0, x_0[1,xy_true], x_0[0,xy_true] + 1] * (1 - x_d[0,xy_true]) + \
                                u.values[t_0 + 1, x_0[1,xy_true], x_0[0,xy_true] + 1] * x_d[0,xy_true]
        
        x_true = np.logical_and(x_0[0,:] < dims[2], x_0[0,:] > -1)
        y_true = np.logical_and(x_0[1,:] < dims[1]-1, x_0[1,:] > -2)  
        xy_true = np.logical_and(x_true, y_true)
        
        c10 = np.zeros(X.shape[1])
        c10[xy_true] = u.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true]] * (1 - x_d[0,xy_true]) + \
                        u.values[t_0 + 1, x_0[1,xy_true] + 1, x_0[0,xy_true]] * x_d[0,xy_true]
       
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1]-1, x_0[1,:] > -2) 
        xy_true = np.logical_and(x_true, y_true)
        
        c11 = np.zeros(X.shape[1])
        c11[xy_true] = u.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true] + 1] * (1 - x_d[0,xy_true]) + \
                        u.values[t_0 + 1, x_0[1,xy_true] + 1,x_0[0,xy_true] + 1] * x_d[0,xy_true]

        c0 = c00 * (1 - x_d[1,:]) + c10 * x_d[1,:]
        c1 = c01 * (1 - x_d[1,:]) + c11 * x_d[1,:]

        ret[0,:] = (c0 * (1 - t_d) + c1 * t_d)
        
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        x_true = np.logical_and(x_0[0,:] < dims[2], x_0[0,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        
        c00 = np.zeros(X.shape[1])
        c00[xy_true] = v.values[t_0, x_0[1,xy_true], x_0[0,xy_true]] * (1 - x_d[0,xy_true]) + \
                            v.values[t_0 + 1, x_0[1,xy_true], x_0[0,xy_true]] * x_d[0,xy_true]
        
                      
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        
        c01 = np.zeros(X.shape[1])
        c01[xy_true] = v.values[t_0, x_0[1,xy_true], x_0[0,xy_true] + 1] * (1 - x_d[0,xy_true]) + \
                                v.values[t_0 + 1, x_0[1,xy_true], x_0[0,xy_true] + 1] * x_d[0,xy_true]
       
        x_true = np.logical_and(x_0[0,:] < dims[2], x_0[0,:] > -1)
        y_true = np.logical_and(x_0[1,:] < dims[1]-1, x_0[1,:] > -2)  
        xy_true = np.logical_and(x_true, y_true)
        
        c10 = np.zeros(X.shape[1])
        c10[xy_true] = v.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true]] * (1 - x_d[0,xy_true]) + \
                        v.values[t_0 + 1, x_0[1,xy_true] + 1, x_0[0,xy_true]] * x_d[0,xy_true]

        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1]-1, x_0[1,:] > -2) 
        xy_true = np.logical_and(x_true, y_true)
        
        c11 = np.zeros(X.shape[1])
        c11[xy_true] = v.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true] + 1] * (1 - x_d[0,xy_true]) + \
                        v.values[t_0 + 1, x_0[1,xy_true] + 1,x_0[0,xy_true] + 1] * x_d[0,xy_true]

        c0 = c00 * (1 - x_d[1,:]) + c10 * x_d[1,:]
        c1 = c01 * (1 - x_d[1,:]) + c11 * x_d[1,:]
        

        ret[1,:] = (c0 * (1 - t_d) + c1 * t_d)
        ret = ret.reshape(2, SHAPE[1], SHAPE[2])
        return ret
# @njit(parallel=True)
# def doublegyre(X, t, A, e, w):
#     a = e * np.sin(w*t)
#     b = 1 - 2.*e*np.sin(w*t)
#     f = a*X[0,:]**2 + b*X[0,:]
#     v = np.empty(X.shape)
#     v[0,:] = -np.pi*A*np.sin(np.pi*f) * np.cos(np.pi*X[1,:])                    # x component of velocity
#     v[1,:] =  np.pi*A*np.cos(np.pi*f) * np.sin(np.pi*X[1,:]) * (2.*a*X[0,:] + b) # y component of velocity
#     return v


# # Wrapper function to pass to integrator
# # X0 is a two-component vector [x, y]
# @njit
# def f(X, t):
#     # Parameters of the velocity field
#     A = A_       # A
#     e = e_      # epsilon
#     w = 2.*np.pi/10.  # omega
#     return doublegyre(X, t, A, e, w)



In [9]:

def f_in2(X, t, offsets, x_res, y_res, time_res, u, v, dims, xcord=None, ycord=None):
    #global offsets, u, v
        SHAPE = X.shape
        #print(SHAPE)
        #X = X.reshape(2, SHAPE[1]*SHAPE[2])
        #X.reshape(2, )
        x_0 = np.zeros(X.shape)
        x_0[0,:] = (((X[0,:] - offsets[0]) / x_res).astype(int))
        x_0[1,:] = (((X[1,:] - offsets[1]) / y_res).astype(int))
        x_0 = x_0.astype(int)
        t_0 = int(t/time_res)
        
        x_d = np.zeros(X.shape)
        x_d[0,:] = ((X[0,:] - offsets[0]) / x_res - x_0[0,:])
        x_d[1,:] = ((X[1,:] - offsets[1]) / y_res - x_0[1,:])
        t_d = t/time_res - t_0

        ret = np.zeros((X.shape))
        
        f00 = np.zeros((X.shape[1], X.shape[2]))
        f01 = np.zeros((X.shape[1], X.shape[2]))
        f10 = np.zeros((X.shape[1], X.shape[2]))
        f11 = np.zeros((X.shape[1], X.shape[2]))
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f00[xy_true] = u.values[t_0, x_0[1,xy_true], x_0[0,xy_true]]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f10[xy_true] = u.values[t_0, x_0[1,xy_true], x_0[0,xy_true] + 1]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f01[xy_true] = u.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true]]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f11[xy_true] = u.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true] + 1]
        
        ret[0,:,:] = f00 * (-x_d[0,:,:] + 1) * (-x_d[1, :, :] + 1)\
                    + f10 * (x_d[0,:,:]) * (-x_d[1, :, :] + 1)\
                    + f01 * (-x_d[0,:,:] + 1) * (x_d[1,:,:])\
                    + f11 * (x_d[0,:,:]) * (x_d[1,:,:])
#         ret = ret.reshape(2, SHAPE[1], SHAPE[2])
        f00 = np.zeros((X.shape[1], X.shape[2]))
        f01 = np.zeros((X.shape[1], X.shape[2]))
        f10 = np.zeros((X.shape[1], X.shape[2]))
        f11 = np.zeros((X.shape[1], X.shape[2]))
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f00[xy_true] = v.values[t_0, x_0[1,xy_true], x_0[0,xy_true]]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f10[xy_true] = v.values[t_0, x_0[1,xy_true], x_0[0,xy_true] + 1]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f01[xy_true] = v.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true]]
        
        x_true = np.logical_and(x_0[0,:] < dims[2]-1, x_0[0,:] > -2)
        y_true = np.logical_and(x_0[1,:] < dims[1], x_0[1,:] > -1)
        xy_true = np.logical_and(x_true, y_true)
        f11[xy_true] = v.values[t_0, x_0[1,xy_true] + 1, x_0[0,xy_true] + 1]
        
        ret[1,:,:] = f00 * (-x_d[0,:,:] + 1) * (-x_d[1, :, :] + 1) \
                    + f10 * (x_d[0,:,:]) * (-x_d[1, :, :] + 1)\
                    + f01 * (-x_d[0,:,:] + 1) * (x_d[1,:,:])\
                    + f11 * (x_d[0,:,:]) * (x_d[1,:,:])
    
        return ret

In [15]:
from scipy.interpolate import interp2d

def f_in3(X, t, offsets, x_res, y_res, time_res, u, v, dims, xcord=None, ycord=None):
    t_0 = int(t/time_res)
#     print(ycord.values.shape)
#     print(xcord.values.shape)
#     print( u.values[t_0,:,:].shape)

    interp_ = interp2d(xcord.values, ycord.values, u.values[t_0,:,:], fill_value=0)
    Shape = X.shape
    X = X.reshape(2, X.shape[1] * X.shape[2])
    ret1 = interp_(X[0,:], X[1,:])
    interp_ = interp2d(xcord.values, ycord.values, v.values[t_0,:,:], fill_value=0)
    ret2 = interp_(X[0,:], X[1,:])
    X = X.reshape(Shape)
    ret1 = ret1.reshape(Shape[1], Shape[2])
    ret2 = ret2.reshape(Shape[1], Shape[2])
    return np.array([ret1, ret2])

In [16]:
from scipy.interpolate import RectBivariateSpline as RBS

def f_in4(X, t, offsets, x_res, y_res, time_res, u, v, dims, xcord=None, ycord=None):
    t_0 = int(t/time_res)
#     print(ycord.values.shape)
#     print(xcord.values.shape)
#     print( u.values[t_0,:,:].shape)
    interp_ = RBS(ycord.values, xcord.values, u.values[t_0,:,:], kx=3, ky=3)
    Shape = X.shape
    X = X.reshape(2, X.shape[1] * X.shape[2])
    ret1 = interp_.ev(X[1,:], X[0,:])
    interp_ = RBS(ycord.values, xcord.values, v.values[t_0,:,:], kx=3, ky=3)
    ret2 = interp_.ev(X[1,:], X[0,:])
    X = X.reshape(Shape)
    ret1 = ret1.reshape(Shape[1], Shape[2])
    ret2 = ret2.reshape(Shape[1], Shape[2])
    return np.array([ret1, ret2])

In [17]:
nork_obj = norkystHandler()
nork_obj(nx, ny)
X0m = nork_obj.grid_of_particles()
# X0m[0, :, 0]
nork_obj.dims

Nx = nx
Ny = ny


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:76: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:77: DeprecationWarning: object of type <class 'numpy.float64'> cannot be safely interpreted as an integer.


In [18]:
def rk4(X, t, h, f):
    k1 = f(X,          t)
    k2 = f(X + k1*h/2, t + h/2)
    k3 = f(X + k2*h/2, t + h/2)
    k4 = f(X + k3*h,   t + h)
    return X + h*(k1 + 2.*k2 + 2.*k3 + k4) / 6.


# Function to calculate endpoints from
# initial positions X0 at t = 0, moving forward
# until t = tmax, using the given timestep and
# integrator

def endpoints(X0, tmax, h, integrator, f, t=0):
    # Number of timesteps
    Nt = int((tmax - t) / h)
    # Array to hold positions
    X  = np.zeros((*X0.shape,))
    # Initial position
    X[:,:] = X0
    # Loop over all timesteps
    i = 1
    
    while t < tmax:
        # Ensure we end up at correct time
        h = min(h, tmax - t)
        X = integrator(X, t, h, f)
        t += h
        i += 1
    # Return entire trajectory
    return X

In [ ]:
# Auxiliary grid initial points
dxa        = 1e-5#dxm / 500
dya        = 1e-5#dym / 500
X0a        = np.empty((2, 2, 2, Nx, Ny))
X1a        = np.empty((2, 2, 2, Nx, Ny))
X0a[0,0,:] = X0m + np.array([-dxa, 0]).reshape(2,1,1)
X0a[1,0,:] = X0m + np.array([ dxa, 0]).reshape(2,1,1)
X0a[0,1,:] = X0m + np.array([ 0,-dya]).reshape(2,1,1)
X0a[1,1,:] = X0m + np.array([ 0, dya]).reshape(2,1,1)

f = nork_obj.f
X1m   = endpoints(X0m, tmax, h, rk4, f, t=0)
X1a[0,0,:] = endpoints(X0a[0,0,:], tmax, h, rk4, f, t=0)
X1a[1,0,:] = endpoints(X0a[1,0,:], tmax, h, rk4, f, t=0)
X1a[0,1,:] = endpoints(X0a[0,1,:], tmax, h, rk4, f, t=0)
X1a[1,1,:] = endpoints(X0a[1,1,:], tmax, h, rk4, f, t=0)

In [ ]:


dxm = X0m[0,1,0] - X0m[0,0,0]
dym = X0m[1,0,1] - X0m[1,0,0]

# Arrays to hold derivatives for original array and auxiliary array
dFMm = np.ones((Nx, Ny, 2, 2))
dFMa = np.ones((Nx, Ny, 2, 2))

#### Derivatives for original array ####
# dFMm_x/dx
dFMm[1:-1, :  ,0,0] = (   X1m[0,2:  , :  ] -   X1m[0, :-2, :  ] )                     / (2*dxm) # Central finite difference
dFMm[   0, :  ,0,0] = ( 4*X1m[0,   1, :  ] - 3*X1m[0,   0, :  ] - 2*X1m[0,   2, :  ]) / (2*dxm)  # One-sided finite difference
dFMm[  -1, :  ,0,0] = ( 3*X1m[0,  -1, :  ] - 4*X1m[0,  -2, :  ] + 2*X1m[0,  -3, :  ]) / (2*dxm)  # One-sided finite difference
# dFMm_x/dy
dFMm[ :  ,1:-1,0,1] = (   X1m[0, :  , 2: ] -   X1m[0, :  , :-2] )                     / (2*dym) # Central finite difference
dFMm[ :  ,   0,0,1] = ( 4*X1m[0, :  ,   1] - 3*X1m[0, :  ,   0] - 2*X1m[0, :  ,   2]) / (2*dym)  # One-sided finite difference
dFMm[ :  ,  -1,0,1] = ( 3*X1m[0, :  ,  -1] - 4*X1m[0, :  ,  -2] + 2*X1m[0, :  ,  -3]) / (2*dym)  # One-sided finite difference
# dFMm_y/dx
dFMm[1:-1, :  ,1,0] = (   X1m[1,2:  , :  ] -   X1m[1, :-2, :  ] )                     / (2*dxm) # Central finite difference
dFMm[   0, :  ,1,0] = ( 4*X1m[1,   1, :  ] - 3*X1m[1,   0, :  ] - 2*X1m[1,   2, :  ]) / (2*dxm)  # One-sided finite difference
dFMm[  -1, :  ,1,0] = ( 3*X1m[1,  -1, :  ] - 4*X1m[1,  -2, :  ] + 2*X1m[1,  -3, :  ]) / (2*dxm)  # One-sided finite difference
# dFMm_y/dy
dFMm[ :  ,1:-1,1,1] = (   X1m[1, :  , 2: ] -   X1m[1, :  , :-2] )                      / (2*dym) # Central finite difference
dFMm[ :  ,   0,1,1] = ( 4*X1m[1, :  ,   1] - 3*X1m[1, :  ,   0] - 2*X1m[1, :  ,   2] ) / (2*dym)  # One-sided finite difference
dFMm[ :  ,  -1,1,1] = ( 3*X1m[1, :  ,  -1] - 4*X1m[1, :  ,  -2] + 2*X1m[1, :  ,   -3]) / (2*dym)  # One-sided finite difference

#### Derivatives for auxiliary array ####
# dx/dx
#FMa -> X1a
dFMa[:,:,0,0] = ( X1a[0,0,0,:,:] - X1a[1,0,0,:,:] ) / (2*dxa) # Central finite difference
# dx/dy
dFMa[:,:,0,1] = ( X1a[0,1,0,:,:] - X1a[1,1,0,:,:] ) / (2*dya) # Central finite difference
# dx/dx
dFMa[:,:,1,0] = ( X1a[0,0,1,:,:] - X1a[1,0,1,:,:] ) / (2*dxa) # Central finite difference
# dx/dy
dFMa[:,:,1,1] = ( X1a[0,1,1,:,:] - X1a[1,1,1,:,:] ) / (2*dya) # Central finite difference

CGm = np.matmul(np.transpose(dFMm, axes = (0,1,3,2)), dFMm)
CGa = np.matmul(np.transpose(dFMa, axes = (0,1,3,2)), dFMa)

vals_m, vecs_m = np.linalg.eigh(CGm)
vals_a, vecs_a = np.linalg.eigh(CGa)
# Store values and vectors separately for less indexing hassle
lambda1 = vals_m[:,:,0]
lambda2 = vals_m[:,:,1]
xi1     = vecs_a[:,:,:,0]
xi2     = vecs_a[:,:,:,1]


In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
fig = plt.figure(figsize = (14,7))

mesh = plt.pcolormesh(X0m[0,:,:], X0m[1,:,:], np.log(lambda2)/(1*tmax))

# plt.savefig('ftle_field' + 'testing' + '.png', dpi=200)


In [ ]:
print(X0m.shape)
print(lambda2.shape)
print(resY)
print(resX)

In [ ]:

Atol = 0.0
Atrue = np.logical_and(np.abs(lambda1 - lambda2) > Atol, lambda2 - 1. > Atol)
#Atrue = np.logical_and(lambda2 == lambda2, lambda2 == lambda2)
print('Number of points where A is true: ', np.sum(Atrue))



lmbd = lambda2.reshape((Nx,Ny))
    
hessian = np.zeros((Nx,Ny,2,2))
    
# First, the laplacian terms:
    
# Internal points: Centered differences
hessian[1:-1,:,0,0] = (lmbd[2:,:]-2*lmbd[1:-1,:]+lmbd[:-2,:])/(dxm**2)
hessian[:,1:-1,1,1] = (lmbd[:,2:]-2*lmbd[:,1:-1]+lmbd[:,:-2])/(dym**2)
    
    # Near boundaries: Higher order forwards/backwards differences
hessian[0,:,0,0] = (2*lmbd[0,:]-5*lmbd[1,:]+4*lmbd[2,:]-lmbd[3,:])/(dxm**2)
hessian[-1,:,0,0] = (2*lmbd[-1,:]-5*lmbd[-2,:]+4*lmbd[-3,:]-lmbd[-4,:])/(dxm**2)
    
hessian[:,0,1,1] = (2*lmbd[:,0]-5*lmbd[:,1]+4*lmbd[:,2]-lmbd[:,3])/(dym**2)
hessian[:,-1,1,1] = (2*lmbd[:,-1]-5*lmbd[:,-2]+4*lmbd[:,-3]-lmbd[:,-4])/(dym**2)

# Then, the mixed derivatives:
dldx = np.empty((Nx,Ny))
dldy = np.empty((Nx,Ny))

# Internal points: Centered differences
dldx[1:-1,:] = (lmbd[2:,:]-lmbd[:-2,:])/(2*dxm)
dldy[:,1:-1] = (lmbd[:,2:]-lmbd[:,:-2])/(2*dym)

# Near boundaries: Higher order forwards/backwards differences
dldx[0,:] = (-3*lmbd[0,:]+4*lmbd[1,:]-lmbd[2,:])/(2*dxm)
dldx[-1,:] = (3*lmbd[-1,:]-4*lmbd[-2,:]+lmbd[-3,:])/(2*dxm)

dldy[:,0] = (-3*lmbd[:,0]+4*lmbd[:,1]-lmbd[:,2])/(2*dym)
dldy[:,-1] = (3*lmbd[:,-1]-4*lmbd[:,-2]+lmbd[:,-3])/(2*dym)



# Same deal for the mixed derivatives
d2ldxdy = np.empty((Nx,Ny))
d2ldydx = np.empty((Nx,Ny))

# Internal points: Centered differences
d2ldxdy[1:-1,:] = (dldy[2:,:]-dldy[:-2,:])/(2*dxm)
d2ldydx[:,1:-1] = (dldx[:,2:]-dldx[:,:-2])/(2*dym)

# Near boundaries: Higher order forwards/backwards differences
d2ldxdy[0,:] = (-3*dldy[0,:]+4*dldy[1,:]-dldy[2,:])/(2*dxm)
d2ldxdy[-1,:] = (3*dldy[-1,:]-4*dldy[-2,:]+dldy[3,:])/(2*dxm)

d2ldydx[:,0] = (-3*dldx[:,0]+4*dldx[:,1]-dldx[:,2])/(2*dym)
d2ldydx[:,-1] = (3*dldx[:,-1]-4*dldx[:,-2]+dldx[:,-3])/(2*dym)

hessian[:,:,0,1] = d2ldxdy
hessian[:,:,1,0] = d2ldydx

hessian = hessian[:,:,:,:].reshape(((Nx)*(Ny), 2, 2))
xi2_m = xi2.reshape((Nx*Ny), 2)
# Taking inner product, although it seems useless
#inner   = np.sum(xi1 * (hessian)[:,:,0] * xi1, axis = -1)
inner = np.sum(xi2_m * np.sum(hessian*xi2_m[...,np.newaxis], axis=1), axis=1)
Btol = 0.0
inner = inner.reshape((Nx,Ny))
Btrue   = inner  <= Btol
print('Number of points where B is true: ', np.sum(Btrue))

ABtrue = np.logical_and(Atrue, Btrue)

print('Number of points where A and B is true: ', np.sum(ABtrue))

In [ ]:
# lambda2 = np.copy(temp)
# xi1 = np.copy(temp_vec1)
# xi2 = np.copy(temp_vec2)
# ABtrue = np.where(ABtrue, 0, 1)
# fig = plt.figure(figsize = (8, 8))
# ax = fig.add_subplot(111, aspect = 'equal')
# plt.pcolormesh(X0m[0,:,:], X0m[1,:,:], Atrue, cmap = plt.get_cmap('Greys'))

# fig = plt.figure(figsize = (8, 8))
# ax = fig.add_subplot(111, aspect = 'equal')
# plt.pcolormesh(X0m[0,:,:], X0m[1,:,:], Btrue, cmap = plt.get_cmap('Greys'))
# ABtrue = median_filter(ABtrue.astype(np.float64), 4)
fig = plt.figure(figsize = (10, 10))
ax = fig.add_subplot(111, aspect = 'equal')
plt.pcolormesh(X0m[0,:,:], X0m[1,:,:], ABtrue, cmap = plt.get_cmap('Greys'))
# plt.savefig('out/ab_dom' + fig_num + '.png')

fig = plt.figure(figsize = (6, 6))
ax = fig.add_subplot(111, aspect = 'equal')
plt.pcolormesh(X0m[0,:,:], X0m[1,:,:], ABtrue, cmap = plt.get_cmap('Greys'))

plt.xlim([-1650000, -1600000])
plt.ylim([-1100000, -1150000])


In [ ]:
fig = plt.figure(figsize=(10,10))
print(xi1.shape)
plt.pcolormesh(xi1[:,:,0].T)
# plt.xlim([430, 450])
# plt.ylim([200, 300])

fig = plt.figure(figsize=(10,10))

plt.pcolormesh(xi2[:,:,0].T)
# plt.savefig('egeinfield_%1.5f_.png' % dxa)
# fig = plt.figure(figsize=(10,10))

# plt.pcolormesh(median_filter(xi2[:,:,0],3))
# print(xi1[435:440, 200,0])